In [62]:
import pandas as pd
import numpy as np
from sklearn import datasets
import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp

In [63]:
file = 'source_data/zumacin_scrubbed.csv'

In [64]:
df = pd.read_csv(file, encoding="ISO-8859-1")

In [65]:
df.head(5)

,SAMPLE_NUMBER,Assay Date,Compartment Temp,Reagent-A ID,Reagent-A ID Exp Date,Reagent-B Exp Date,Reagent-B ID,Material-A ID,Reagent-C Lot,Reagent-D Lot,Reagent-E Lot,Reagent-F Concentration,Reagent-F Expiration,Reagent-F Lot,SOP Version,Agg%,INSTRUMENT,Main%,Lab_Site,Chemist
0,120629,01/28/2011,4.3,ReagentA-1-32,04/25/2011,NaN,NaN,MaterialA-1-554,NaN,NaN,ReagentE-1-394,NaN,01/31/2012,ReagentF-1-779,5.6,5.90580,30.0,88.62140,Stat,ChemistO
1,126733,02/15/2011,4.3,ReagentA-2-654,05/12/2011,02/23/2011,ReagentB-2-867,MaterialA-2-47,NaN,NaN,ReagentE-1-394,NaN,01/31/2012,ReagentF-1-779,5.6,6.09960,84.0,88.35680,Park,ChemistV
2,134380,03/04/2011,4.6,ReagentA-1-32,04/25/2011,NaN,NaN,MaterialA-3-451,NaN,NaN,ReagentE-2-800,NaN,01/31/2012,ReagentF-1-779,5.6,5.92722,37.0,88.49498,Stat,ChemistO
3,140369,03/23/2011,4.3,ReagentA-3-383,06/14/2011,03/30/2011,ReagentB-3-584,MaterialA-2-47,NaN,NaN,ReagentE-1-394,NaN,01/31/2012,ReagentF-1-779,5.6,6.05880,53.0,88.39796,Park,ChemistV
4,142003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
df.groupby(['Reagent-F Lot']).count()

,SAMPLE_NUMBER,Assay Date,Compartment Temp,Reagent-A ID,Reagent-A ID Exp Date,Reagent-B Exp Date,Reagent-B ID,Material-A ID,Reagent-C Lot,Reagent-D Lot,Reagent-E Lot,Reagent-F Concentration,Reagent-F Expiration,SOP Version,Agg%,INSTRUMENT,Main%,Lab_Site,Chemist
Reagent-F Lot,,,,,,,,,,,,,,,,,,,
ReagentF-1-779,133,133,121,133,133,17,17,132,116,121,133,121,133,132,119,133,122,133,133
ReagentF-3-623,188,188,186,187,187,0,0,184,0,187,177,188,188,188,147,188,149,188,188
ReagentF-4-825,95,95,91,95,95,0,0,95,0,95,95,95,95,95,91,95,91,95,95


In [67]:
df['Reagent-C Lot'].unique()

array([nan, 'ReagentC-2-979', 'ReagentC-3-410'], dtype=object)

Items to include in the analysis:
- Reagent-F lot
- Compartment Temp
- Reagent-A ID
- Material-A ID
- Reagent-D Lot
- SOP Version
- INSTRUMENT
- Lab_Site
- Chemist

In [68]:
an_df = df[['Reagent-F Lot', 'Compartment Temp', 'Reagent-A ID', 'Material-A ID', 'Reagent-D Lot', 'SOP Version', 'INSTRUMENT',
           'Lab_Site', 'Chemist','Agg%','Main%']]

In [69]:
an_df.head()

,Reagent-F Lot,Compartment Temp,Reagent-A ID,Material-A ID,Reagent-D Lot,SOP Version,INSTRUMENT,Lab_Site,Chemist,Agg%,Main%
0,ReagentF-1-779,4.3,ReagentA-1-32,MaterialA-1-554,NaN,5.6,30.0,Stat,ChemistO,5.90580,88.62140
1,ReagentF-1-779,4.3,ReagentA-2-654,MaterialA-2-47,NaN,5.6,84.0,Park,ChemistV,6.09960,88.35680
2,ReagentF-1-779,4.6,ReagentA-1-32,MaterialA-3-451,NaN,5.6,37.0,Stat,ChemistO,5.92722,88.49498
3,ReagentF-1-779,4.3,ReagentA-3-383,MaterialA-2-47,NaN,5.6,53.0,Park,ChemistV,6.05880,88.39796
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
final_df = an_df.dropna(how='any')

In [71]:
final_scrubbed = final_df.rename(columns={'Agg%':'Agg'})

In [72]:
final_scrubbed.head()

,Reagent-F Lot,Compartment Temp,Reagent-A ID,Material-A ID,Reagent-D Lot,SOP Version,INSTRUMENT,Lab_Site,Chemist,Agg,Main%
17,ReagentF-1-779,4.3,ReagentA-8-484,MaterialA-5-420,ReagentD-2-446,6.6,32.0,Stat,ChemistB,6.25617,87.84769
19,ReagentF-1-779,4.3,ReagentA-10-950,MaterialA-5-420,ReagentD-2-446,6.6,29.0,Stat,ChemistB,6.12357,88.01086
24,ReagentF-1-779,4.6,ReagentA-7-462,MaterialA-7-511,ReagentD-4-395,6.6,83.0,Park,ChemistW,6.02463,88.25194
25,ReagentF-1-779,4.3,ReagentA-14-651,MaterialA-5-420,ReagentD-3-403,6.6,29.0,Stat,ChemistB,6.07512,88.12062
26,ReagentF-1-779,4.3,ReagentA-15-112,MaterialA-5-420,ReagentD-3-403,6.6,29.0,Stat,ChemistO,6.01953,88.19020


In [73]:
final_scrubbed.groupby(['Reagent-F Lot']).count()

,Compartment Temp,Reagent-A ID,Material-A ID,Reagent-D Lot,SOP Version,INSTRUMENT,Lab_Site,Chemist,Agg,Main%
Reagent-F Lot,,,,,,,,,,
ReagentF-1-779,99,99,99,99,99,99,99,99,99,99
ReagentF-3-623,144,144,144,144,144,144,144,144,144,144
ReagentF-4-825,87,87,87,87,87,87,87,87,87,87


In [74]:
final_scrubbed.reset_index(drop=True, inplace=True)

In [75]:
rp.summary_cont(final_scrubbed['Agg'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,Agg,330.0,3.572817,1.688248,0.092935,3.389995,3.755639


In [76]:
rp.summary_cont(final_scrubbed.groupby(['Lab_Site']))['Agg']

,N,Mean,SD,SE,95% Conf.,Interval
Lab_Site,,,,,,
LIN,81,2.561109,0.381488,0.042388,2.478030,2.644189
Park,12,6.169719,0.183729,0.053038,6.065764,6.273673
Stat,237,3.787102,1.783848,0.115873,3.559990,4.014214


In [77]:
rp.summary_cont(final_scrubbed.groupby(['Reagent-F Lot', 'Lab_Site']))['Agg']

N      Mean        SD        SE  95% Conf.  Interval
Reagent-F Lot  Lab_Site                                                       
ReagentF-1-779 Park      12  6.169719  0.183729  0.053038   6.065764  6.273673
               Stat      87  6.079796  0.246526  0.026430   6.027992  6.131599
ReagentF-3-623 LIN       46  2.246042  0.150152  0.022139   2.202650  2.289434
               Stat      98  2.181948  0.128767  0.013007   2.156454  2.207443
ReagentF-4-825 LIN       35  2.975197  0.042143  0.007124   2.961235  2.989159
               Stat      52  2.976346  0.054852  0.007607   2.961437  2.991255

In [78]:
final_scrubbed.rename(columns={'Reagent-F Lot':'ReagentF'}, inplace=True)

In [79]:
final_scrubbed.rename(columns={'SOP Version':'SOP_V'}, inplace=True)

In [80]:
rp.summary_cont(final_scrubbed.groupby(['ReagentF', 'Lab_Site']))['Agg']

N      Mean        SD        SE  95% Conf.  Interval
ReagentF       Lab_Site                                                       
ReagentF-1-779 Park      12  6.169719  0.183729  0.053038   6.065764  6.273673
               Stat      87  6.079796  0.246526  0.026430   6.027992  6.131599
ReagentF-3-623 LIN       46  2.246042  0.150152  0.022139   2.202650  2.289434
               Stat      98  2.181948  0.128767  0.013007   2.156454  2.207443
ReagentF-4-825 LIN       35  2.975197  0.042143  0.007124   2.961235  2.989159
               Stat      52  2.976346  0.054852  0.007607   2.961437  2.991255

In [81]:
final_scrubbed['INSTRUMENT'] = final_scrubbed['INSTRUMENT'].astype(str)

In [82]:
model2 = ols('Agg ~ C(ReagentF) + C(Lab_Site)', final_scrubbed).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

Overall model F( 4, 325) =  8894.713, p =  0.0000


In [83]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    Agg   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                     8895.
Date:                Mon, 20 May 2019   Prob (F-statistic):               0.00
Time:                        21:09:37   Log-Likelihood:                 135.72
No. Observations:                 330   AIC:                            -261.4
Df Residuals:                     325   BIC:                            -242.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         6.1178      0.028    216.256      0.000       6.062       6.173
C(ReagentF)[T.ReagentF-3-623]    -3.8895      0.023   -168.536      0.000      -3.935      -3.844
C(ReagentF)[T.ReagentF-4-825]    -3.1192      0.026   -119.498      0.000      -3.171      -3.068
C(Lab_Site)[T.Park]               0.0520      0.055      0.952      0.342      -0.055       0.159
C(Lab_Site)[T.Stat]              -0.0380      0.022     -1.697      0.091      -0.082       0.006
==============================================================================
Omnibus:                       43.343   Durbin-Watson:                   1.409
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              161.969
Skew:                          -0.488   Prob(JB):                     6.74e-36
Kurtosis:                       6.290   Cond. No.                         9.27
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The Durban-Watson test is to detect the presence of autocorrelation, Jarque-Bera tests the assumption of normality, Omnibus tests the assumption of homogeneity of variance, and the Condition Number assess multicollinearity. Condition Number values over 20 are indicative of multicollinearity.

In [84]:
results = sm.stats.anova_lm(model2, typ= 2)
results

,sum_sq,df,F,PR(>F)
C(ReagentF),754.504754,2.0,14444.565845,3.328026e-318
C(Lab_Site),0.160525,2.0,3.073166,4.762166e-02
Residual,8.488107,325.0,NaN,NaN
